# Spam Detection Using Spark

## Overview of Spark

Spark is a widely used open-source data processing framework which allows you to distribute your compute power across many servers in lieu of your local machine. It uses Resilient Distributed Datasets (RDDs) as it's foundational data structure. RDDs are a distributed collection of objects which can be divided into logical partitions operated on in parallel across a cluster of computers. Here are some key features of RDDs:

1. Resilience: RDDs can recover lost data caused by the failure of a node. This is also called fault tolerance.
2. Distributed: data present in an RDD resides on multiple nodes of a cluster.
3. Lazy evaluation: transformations are performed on data on an as-needed basis.
4. Immutability: data stored in an RDD is in the read-only mode and is unchangeable over time.
5. In-memory: it is possible to store data in an RDDs and perform in-memory computation on the data.
6. Partitioning: data in RDDs can be grouped into logical parts, called partitions. Data transformations can be applied on these partitions. 

Spark provides a high-level API for Python, i.e. pyspark.

In [20]:
!pip install pyspark
from pyspark import SparkContext
import os
from pathlib import Path
import re
!spark-submit --version
import nltk
import re
from nltk.corpus import stopwords
from pyspark.mllib.feature import IDF, Normalizer
from pyspark.mllib.classification import (NaiveBayes, LogisticRegressionWithLBFGS, SVMWithSGD) 
import numpy
from pyspark.mllib.regression import LabeledPoint
from time import time
from pyspark.sql import DataFrame
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter
import string

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 2.4.5
      /_/
                        
Using Scala version 2.11.12, Java HotSpot(TM) 64-Bit Server VM, 1.8.0_121
Branch HEAD
Compiled by user centos on 2020-02-02T19:38:06Z
Revision cee4ecbb16917fa85f02c635925e2687400aa56b
Url https://gitbox.apache.org/repos/asf/spark.git
Type --help for more information.


## Objectives

Natural language processing (NLP) is the application of machine learning algorithms to teach computers how to understand human language. This project will build and evaluate NLP models that classify email messages as spam or non-spam. A Spark data pipeline will be used to train and evaluate these spam classifiers. Python's Natural Language Toolkit (NLTK) will be used to perform some of the data processing steps. 

Machine learning models cannot make predictions using raw text alone - they require numerical values to perform the training computations. In this report, we will use a method called Term Frequency Inverse Data Frequency (TF-IDF) to convert text to numbers for our NLP models. 

## Dataset

We will use the lingspam dataset in this project to build the spam classifiers.

In [2]:
os.chdir('datasets/')
print("Extracting the ling_spam dataset, this can take a moment.")
!tar -xf lingspam_public02.tar.gz
print(">>> Unzipping finished.")

Extracting the ling_spam dataset, this can take a moment.
>>> Unzipping finished.


After unzipping the files, you will notice that there are 4 sub-directories corresponding to the 4 versions of the corpus:
1. bare: lemmatiser disabled, stop-list disabled.
2. lemm: lemmatiser enabled, stop-list disabled.
3. lemm_stop: lemmatiser enabled, stop-list enabled.
4. stop: lemmatiser disabled, stop-list enabled.

Each one of these 4 directories contain 10 sub-directories (part1, 
..., part10). These correspond to the 10 partitions of the corpus 
that were used in the 10-fold experiments. In each repetition, one 
part was reserved for testing and the other 9 were used for training. 

Each one of the 10 subdirectories contain both spam and legitimate 
messages, one message in each file. Files whose names have the form
spmsg*.txt are spam messages. All other files are legitimate messages.

<u>Definitions</u>
 - corpus: a collection of texts
 - lemmatization: the grouping of words that share the same structure but are spelled differently, e.g. "organize, organizes, organizing"
 - stop list: list of stop words, which are commonly used words that are filtered out for text processing given their lack of predictive power, e.g. "a," "and," "but," "how," "or," and "what"

## Data Processing 

### Read the dataset and create RDDs

Now that the data has been loaded, the first step is to read the dataset and create RDDs. To do so, a Spark context must be created, which represents the connection to a Spark cluster. 

In [3]:
sc = SparkContext("local")

The next steps to create the RDDs are:
1. Start by reading the directory with text files from the file system ("datasets/bare"). Load all text files per directory (part1, part2, ... , part10) using `wholeTextFiles()`, which creates one RDD per part, containing tuples (filename, text). This is a good choice as the text files are small.
2. We will use one of the RDDs as test set, the rest as training set. For the training set you need to create the union of the remaining RDDs.
3. Remove the path and extension from the filename using the regular expression provided in the code snippet below.

The training set labels are "spam" is the file name starts with "spmsg" and "non-spam" otherwise. 

We will put the code in each cell into a function that we can reuse later. In this way we can develop the whole preprocessing with the smaller test set and apply it to the training set once we know that everything works. 

In [4]:
def makeTestTrainRDDs(pathString):
    ''' Takes one of the four subdirectories of the lingspam dataset and returns two RDDs one each for testing 
        and training. We should see 10 parts that we can use for creating train and test sets.
    '''
    p = Path(pathString) # gets a path object representing the current directory path.
    dirs = list(p.iterdir()) # get the directories part1 ... part10. 
    dirs = [str(dir.absolute()) for dir in dirs]
    rddList = [] # create a list for the RDDs
    for d in dirs: # iterate through the directories
        rdd = sc.wholeTextFiles(d)
        rddList.append(rdd)

    testRDD1 = rddList[9] # set the test set
    trainRDD1 = rddList[0] # start the training set from 0 and 
    for i in range(1,9):
        trainRDD1 = trainRDD1.union(rddList[i]) # create a union of the current and the next
        testRDD2 = testRDD1.map(lambda x : (re.split('[/\.]', x[0])[-2],x[1]))
        trainRDD2 = trainRDD1.map(lambda x : (re.split('[/\.]', x[0])[-2],x[1]))
    return (trainRDD2,testRDD2)

We can now print out the number of test and train RDDs that we have created

In [5]:
trainRDD_testRDD = makeTestTrainRDDs('lingspam_public/bare') # read from the 'bare' directory - takes a bit of time
(trainRDD,testRDD) = trainRDD_testRDD # unpack the returned tuple
print('testRDD.count(): ',testRDD.count()) # count of RDDs in the test set
print('trainRDD.count(): ',trainRDD.count(),'\n') # count of RDDs in the training set
print('Example RDD: \n',testRDD.take(1)) # should be (filename,[tokens]) 
rdd1 = testRDD # use this for development in the next tasks 

testRDD.count():  1734
trainRDD.count():  15158 

Example RDD: 
 [('spmsgc104 3', 'Subject: free y2k fix ! ! ! < > . : adv " " . ,\n\nthe good news is : that you can now test your computer for full y2k compliance ( both bios and real time clocks ) and even correct it with a simple , inexpensive download . the great news is : we offer a free test & evaluation period while you decide whether you want to purchase this solution . no one else offers the comprehensive guarantee that we have for you . for additional information on our free evaluation period , reply to : y2kfreetest @ popmail . com type y2k on the subject line * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * to be removed from this mailing list reply to : resource22 @ popmail . com , type remove on subject line . thank you * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *\n'

The `take()` method for the RDD shows us that each email can be thought of to have a key-value pair structure where:
- the key is the email ID, e.g. "8-1041msg1 2", including if it's spam or not ("spmsg" or "msg")
- the value is the email itself, including the subject and body

### Tokenize and remove punctuation

Now we need to split the words, a process called *tokenization* by linguists, and remove punctuation. We will use the Python [Natural Language Toolkit](http://www.nltk.org) *NLTK* to do the tokenization. We use the NLTK function `word_tokenize`. Then we will remove punctuation. There is no specific function for this, so we use a regular expression in a list comprehension. 

We use a new technique here: we separate keys and values of the RDD, using the RDD functions `keys()` and `values()`, which yield each a new RDD. Then we process the values and *zip* them together with the keys again. We wrap the whole sequence into one function `prepareTokenRDD` for later use.

In [6]:
def tokenize(text):
    ''' Apply the nltk.word_tokenize() method to our text, return the token list. 
        It is important that this is done here in the function, as it needs to be done on every worker.
        If we do the download outside a this function, it would only be executed on the driver 
    '''
    nltk.download('punkt') # this loads the standard NLTK tokenizer model er     
    return nltk.word_tokenize(text)
    
def removePunctuation(tokens):
    ''' Remove punctuation characters from all tokens in a provided list. 
        This will remove all punctiation from string
    '''
    tokens2 = [re.sub('[()\[\],.?!";_]','',i) for i in tokens ]
    return tokens2

def prepareTokenRDD(fn_txt_RDD):
    ''' Take an RDD with (filename,text) elements and transform it into a (filename,[token ...]) 
        RDD without punctuation characters.  We zip the two RDDs together i.e. produce tuples with one item
        from each RDD. This works because we have only applied mappings to the values, therefore the items in both 
        RDDs are still aligned.
    '''
    rdd_vals2 = fn_txt_RDD.values() # it's convenient to process only the values. 
    rdd_vals3 = rdd_vals2.map(tokenize) # create a tokenised version of the values by mapping
    rdd_vals4 = rdd_vals3.map(removePunctuation) # remove punctuation from the values
    rdd4 = fn_txt_RDD.keys().zip(rdd_vals4) # zip RDDs together
    rdd5 = rdd4.map(lambda x: (x[0],[i for i in x[1] if len(i)!=0]))    
    rdd6 = rdd5.filter(lambda x: len(x[1])!=0) # remove items without tokens using RDD.filter and a lambda. 
    return rdd6 

By printing a resulting RDD, we now see that the email has been tokenized: each word or punctuation mark is a single string item (token) in a list.

In [7]:
rdd2 = prepareTokenRDD(rdd1) # Use the test set for now, because it is smaller
print(rdd2.take(1)[0]) # for checking result

('spmsgc104 3', ['Subject', ':', 'free', 'y2k', 'fix', '<', '>', ':', 'adv', '``', '``', 'the', 'good', 'news', 'is', ':', 'that', 'you', 'can', 'now', 'test', 'your', 'computer', 'for', 'full', 'y2k', 'compliance', 'both', 'bios', 'and', 'real', 'time', 'clocks', 'and', 'even', 'correct', 'it', 'with', 'a', 'simple', 'inexpensive', 'download', 'the', 'great', 'news', 'is', ':', 'we', 'offer', 'a', 'free', 'test', '&', 'evaluation', 'period', 'while', 'you', 'decide', 'whether', 'you', 'want', 'to', 'purchase', 'this', 'solution', 'no', 'one', 'else', 'offers', 'the', 'comprehensive', 'guarantee', 'that', 'we', 'have', 'for', 'you', 'for', 'additional', 'information', 'on', 'our', 'free', 'evaluation', 'period', 'reply', 'to', ':', 'y2kfreetest', '@', 'popmail', 'com', 'type', 'y2k', 'on', 'the', 'subject', 'line', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*'

### Creating normalised TF.IDF vectors of defined dimensionality

#### The hashing trick

The emails in the corpus have different lengths hence the training word vectors will vary as well. The hashing function maps data of arbitrary size to a fixed size. Using the hashing trick, each email, no matter the size, will be transformed into a fixed length numerical vector. All hashing functions share the following properties:
 - the hashing function will always produce the same output if given the same input
 - the choice of the hashing function will determine the range of the outputs, e.g. one hashing function might return values which can only fall in the 0-1024 range
 - hashing functions are one-way (input to output) hence you cannot reverse engineer the hashing function to determine the input given the output
 - hashing functions might output the same values for multiple inputs (collision)
 
We create a function called `hashing_vectorize` using the built-in `hash` method to transform all emails into vectors of fixed length `N`. 

In [8]:
# use the hashing trick to create a fixed-size vector from a word list
def hashing_vectorize(text,N): # arguments: the list and the size of the output vector
    v = [0] * N  # create vector of 0s
    for word in text: # iterate through the words 
        h = hash(word)# get the hash value
        v[h%N] += 1 # add 1 at the hashed address
    return v # return hashed word vector

#### Normalized TF-IDF

TF-IDF is a measure that determines how specific a word is within a document. For example, if a word appears many times in a document (term frequency) but does not appear frequently in other documents, then it will have a high TF-IDF score. Common words which are used frequently across multiple documents, or in this case emails, will have a low TF-IDF score. It is calculated my multiplying the following terms together:
 - term frequency (TF): number of times a word appears in the email
 - inverse document frequency (IDF): calculated by taking the logarithm of the total number of emails divided by the number of emails that contain a specific word

We use the TF-IDF values to populated our fixed length hashing vectors by creating a TF-IDF object (`IDF()`) and normalizing the results (`Normalizer()) between 0 and 1.

In [9]:
def normTFIDF(fn_tokens_RDD, vecDim, caching=True):
    keysRDD = fn_tokens_RDD.keys()
    tokensRDD = fn_tokens_RDD.values()
    tfVecRDD = tokensRDD.map(lambda tokens: hashing_vectorize(tokens,vecDim))

    idf = IDF() # create IDF object
    idfModel = idf.fit(tfVecRDD) # calculate IDF values
    tfIdfRDD = idfModel.transform(tfVecRDD)
    # create a Normalizer object like in the example linked above
    norm = Normalizer() # create a Normalizer object like in the example linked above
    # apply Normalizer to the tfIdfRDD
    normTfIdfRDD = norm.transform(tfIdfRDD) # and apply it to the tfIdfRDD
    # zip the keys and values together
    zippedRDD = keysRDD.zip(normTfIdfRDD) # zip the keys and values together
    return zippedRDD

For each email, we now have a tuple consisting of the filename (including the spam classification) as well as a fixed length numerical vector which can be used for model training. We print out one of these vectors below, in the case where the length N is equal to 10. 

In [10]:
testDim = 10 # too small for good accuracy, but OK for testing
rdd3 = normTFIDF(rdd2, testDim, True) # test our
print(rdd3.take(1))

[('spmsgc104 3', DenseVector([0.9896, 0.0324, 0.0255, 0.0, 0.0, 0.0974, 0.0465, 0.037, 0.0767, 0.0]))]


### Side experiment: measure the impact of caching

The `normTFIDF` lets us switch caching on or off. We measure the effect of caching by noting the time for both options.

In [11]:
# run a small experiment with caching set to True or False, 3 times each
resCaching = [] # for storing results
resNoCache = [] # for storing results
for i in range(3): # 3 samples
    # start timer
    startTime = time()
    testRDD1 = normTFIDF(rdd2, testDim, True) # 
    # call an action on the RDD to force execution
    testRDD1.collect()
    # end timer
    endTime = time()
    resCaching.append( endTime - startTime ) # calculate the difference
    
    # start timer
    startTime = time()
    testRDD2 = normTFIDF(rdd2, testDim, False) 
    # call an action to force execution   
    testRDD2.collect()
    # end timer
    endTime = time()
    resNoCache.append(endTime - startTime)

# calculate average times
meanTimeCaching = round(sum(resCaching)/len(resCaching),2)
meanTimeNoCache = round(sum(resNoCache)/len(resCaching),2)

print(' Creating TF.IDF vectors, 3 trials \n Mean time with caching: ',
      meanTimeCaching, 'seconds \n Mean time without caching: ',
      meanTimeNoCache,'seconds')

 Creating TF.IDF vectors, 3 trials 
 Mean time with caching:  71.48 seconds 
 Mean time without caching:  69.59 seconds


The results for this particular output show that caching is faster than non-caching. 

We are calculating wall clock time here (i.e. latency). The results will be noisy, especially for short running jobs such as this one. However, when we run the cell multiple times we consistently see that caching is the faster option.

Usually, the work law states that, given n the number of processors, multiplying n by the time it takes to run the task on n processors has to be greater than the time it takes to run a task on a single processor. However, in practice this might sometimes not be the case and the reason for this is memory hierarchy: if the partitioned data is small enough to fit into memory, then it could be faster than running it on one machine that is reading and writing from disk.  

In addition, non-caching executes different data pipelines independently, whereas these processes can be shared using caching, thus taking less time.

Finally, caching is faster because the RDD is re-used multiple times within the foor loop. The data is cached during the first action call (`.count`). All of the repeat actions will find the RDD blocks in memory. With caching not enabled this is not the case and the task takes longer since spark evaluates lazily and the RDD must be re-evaluated in full each time an action is called.

### Create LabeledPoints

We now create a function that transforms the labels (current example: "8-1041msg1 2") into labels representing a binary classification: 1 if email is spam, 0 otherwise. We use the `RDD.map()` to create an RDD of LabeledPoint objects `LabeledPoint()`. An example LabelPoints for a non-spam email is printed
below.

In [12]:
# create labelled points of vector size N out of an RDD with normalised (filename [(word,count), ...]) items
def makeLabeledPoints(fn_vec_RDD): # RDD and N needed 
    # we determine the true class as encoded in the filename and represent as 1 (spam) or 0 (good)
    # use a conditional expression to get the class label (True or False)
    cls_vec_RDD = fn_vec_RDD.map(lambda x: (1 if x[0].startswith('sp') else 0,x[1]) )
    # now we can create the LabeledPoint objects with (class,vector) arguments
    lp_RDD = cls_vec_RDD.map(lambda cls_vec: LabeledPoint(cls_vec[0],cls_vec[1]))
    return lp_RDD 

# for testing
testLpRDD = makeLabeledPoints(rdd3) 
print(testLpRDD.take(1)) 

[LabeledPoint(1.0, [0.9896457148958359,0.03241063850896587,0.025465501685616038,0.0,0.0,0.09740091687861234,0.04654346854229574,0.03704072972453242,0.07666269622432531,0.0])]


### Consolidate data processing steps

It will be useful to have a single function (`loadAndPreprocess`) that consolidates all the data processing steps. We can tune the parameter N, the vector size, to improve the accuracy of the models.

In [13]:
# now we can apply the preprocessing chain to the loaded data  
# N is for controlling the vector size
def preprocess(rawRDD,N):
    """ take a (filename,text) RDD and transform into LabelledPoint objects 
        with class labels and a TF.IDF vector with N dimensions. 
    """
    tokenRDD = prepareTokenRDD(rawRDD) 
    tfIdfRDD = normTFIDF(tokenRDD,N) 
    lpRDD = makeLabeledPoints(tfIdfRDD) 
    return lpRDD # return RDD with LabeledPoints

# and with this we can start the whole process from a directory, N is again the vector size
def loadAndPreprocess(directory,N):
    """ load lingspam data from a directory and create a training and test set of preprocessed data """
    # read from the directory using the function created 
    trainRDD_testRDD = makeTestTrainRDDs(directory)
    # unpack the returned tuple
    (trainRDD,testRDD) = trainRDD_testRDD
    return (preprocess(trainRDD,N),preprocess(testRDD,N)) # apply the preprocessing function defined above

# prepare the training data
trainLpRDD = preprocess(trainRDD,testDim) 
# let's re-run with another vector size
train_test_LpRDD = loadAndPreprocess('lingspam_public/lemm',100) 
(trainLpRDD,testLpRDD) = train_test_LpRDD 
print(testLpRDD.take(1)) # print test RDD example for N=100

[LabeledPoint(1.0, [0.006820394456764202,0.005252299608125127,0.0,0.00994561206584296,0.0,0.02268149523518462,0.03397488030802731,0.0,0.0,0.0,0.023863445974781305,0.0,0.0,0.0,0.0,0.013913161502230941,0.0,0.009018001325248876,0.0,0.014461444011701827,0.0,0.041181949816747124,0.03544340052598322,0.0,0.014641487650620656,0.014295504642136034,0.0,0.01491841809876444,0.0,0.0,0.05996516094359604,0.0,0.0,0.0,0.0,0.0,0.011655772207896209,0.03449360524237956,0.016800685414711972,0.0,0.9834505093182238,0.0,0.018732126345668537,0.0,0.0,0.0,0.0,0.019027337709104277,0.04764922235122011,0.012620085552194394,0.020828081825915,0.012783696736923226,0.0,0.0,0.019521069009794875,0.033423954935694174,0.020828081825915,0.0,0.04544159896117538,0.012948157118455345,0.0,0.024808190611971637,0.012295376008186124,0.0,0.00907289874653656,0.01535817947925166,0.0,0.0,0.031046686299388535,0.01814579749307312,0.011240723722172382,0.019921313855009025,0.0455029512226766,0.0026261498040625634,0.0,0.0551340482557182,0.

## Model Training

### Train some classifiers: LR, NB and SVM

We can now use the `LabelPoint()` object to train the following classifiers: Logistic Regression (LR), Naive Bayes (NB) and Support Vector Machine (SVM). We calculate the accuracy of the model on the training set. LR is the best performing model. 

In [14]:
## train the model with an (f,[(w,c), ...]) RDD. This is practical as we can reuse the function for TF.IDF
def trainModel(lpRDD):
    '''
       Train 3 classifier models on the given RDD with LabeledPoint objects.
       A list of trained model is returned.
    '''
    # LR - this is the best model
    model1 = LogisticRegressionWithLBFGS.train(lpRDD) 
    
    # NB - doesn't work well
    model2 = NaiveBayes.train(lpRDD) #

    # SVM - doesn't work well
    model3 = SVMWithSGD.train(lpRDD) 
    return [model1,model2,model3]

def testModel(model, lpRDD):
    '''Tests the classification accuracy of the given model on the given RDD with LabeledPoint objects. '''
    # make prediction and evaluate training set accuracy
    # get the prediction and ground truth (label) for each item.
    predictionAndLabel = lpRDD.map(lambda p: (model.predict(p.features), p.label)) 
    
    # count the correct predictions 
    correct = predictionAndLabel.filter(lambda xv: xv[0] == xv[1]).count() 

    # and calculate the accuracy 
    accuracy = correct/lpRDD.count()
    print('Accuracy {:.1%} (data items: {}, correct: {})'.format(accuracy,lpRDD.count(), correct)) 
    return accuracy # and return the value  

models = trainModel(trainLpRDD) # just for testing
print('\nLR \n--')
testModel(models[0], trainLpRDD) # just for testing
print('\nNB \n--')
testModel(models[1], trainLpRDD) # just for testing
print('\nSVM \n---')
testModel(models[2], trainLpRDD) # just for testing
print('\nLR is the best performing model.')


LR 
--
Accuracy 96.9% (data items: 15623, correct: 15143)

NB 
--
Accuracy 83.4% (data items: 15623, correct: 13025)

SVM 
---
Accuracy 83.4% (data items: 15623, correct: 13025)

LR is the best performing model.


### Automate training and testing

We can now automate the whole process from reading the files, through preprocessing, and training up to evaluating the models. In the end we have a single function that takes all the parameters we are interested in and produces trained models and an evaluation. 

In [15]:
# this method should take RDDs with (f,[(w,c), ...])
def trainTestModel(trainRDD,testRDD):
    '''
        Trains 3 models and tests them on training and test data.
        Returns a matrix the training and testing (rows) accuracy values for all models (columns). 
    ''' 
    # train models on the training set
    models = trainModel(trainRDD)
    results = [[],[]] # matrix for 2 modes (training/test) vs n models (currently 3) 
    for mdl in models:
        results[0].append(testModel(mdl,trainRDD))
        results[1].append(testModel(mdl,testRDD))
    return results

def trainTestFolder(folder,N):
    '''
        Reads data from a folder, preproceses the data, and trains and evaluates models on it.
    '''
    train_test_LpRDD = loadAndPreprocess(folder,N) # create the RDDs
    (trainLpRDD,testLpRDD) = train_test_LpRDD # unpack the RDDs 
    return trainTestModel(trainLpRDD,testLpRDD) # train and test

trainTestFolder('lingspam_public/lemm',10)

Accuracy 83.4% (data items: 15623, correct: 13037)
Accuracy 67.1% (data items: 1734, correct: 1164)
Accuracy 83.4% (data items: 15623, correct: 13025)
Accuracy 83.4% (data items: 1734, correct: 1446)
Accuracy 83.4% (data items: 15623, correct: 13025)
Accuracy 83.4% (data items: 1734, correct: 1446)


[[0.8344748127760353, 0.8337067144594508, 0.8337067144594508],
 [0.671280276816609, 0.8339100346020761, 0.8339100346020761]]

## Analysis and Results

### Experiments

We now have a single function that allows us to vary the vector size easily. We will run experiments to test vector sizes 3, 30, 300, 3000 and examine the effect on the classification accuracy.

We can use the function `trainTestFolder` to test different data types. The dataset has raw text in folder `bare`, lemmatised text in  `lemm` (similar to stemming, reduces to basic word forms), `stop` (with stopwords removed), and `lemm_stop` (lemmatised and stopwords removed). We can also test how the classification accuracy differs for these four data types. The results can be collected in a data structure that can be saved for future analysis. The results of the experiments are found in the Appendix at the end of this report.

### Performance when varying vector size

Increasing the size of the density vectors improves the train and test accuracy for the LR and NB classifiers. At smaller vector sizes, we have more collisions, i.e., the tokens (words) are  more often mapped to the same indices. When several collisions occur, the vector value for an index goes to 0 based on the formula log(N/dfi). At these smaller vector sizes we also see comparable results for the 3 classifiers and there is no overfitting. As the  vector size increases, this minimizes the number of collisions.

Using LR, if the vector size is large enough (over 300) then each word can be mapped to an index without any collisions, the training accuracy goes to 100% and there is overfitting on the data. Naive Bayes test accuracy increases up to a vector size of 3000 but collapses at the higher value. This is because Vector length of 30,000 is larger than the  average vector length of emails in the 'bare' corpus, as well as the assumption of independence inherent in NB is not correct for this dataset.

The effects of varying the vector size does not have an effect on the  accuracy of the SVM, which stays constant at 83.4% (train) and 83.2%(test). SVM is also the worst performing model compared to LR and NB. SVM models attempt to minimise overall error and tend to perform poorly on the minority class in unbalanced problems (spam classification is typically highly unbalanced).

### Performance on different data types

The results show that for LR the train and test accuracy remain constant for the different data types (bare, stop, lemm and lemm stop) at 100% (train) and 97.6% (test). It achieved the best results compared to the two other models. SVM also shows similar results (83.4% for training and 83.2% for testing) regardless if the words are transformed or not, via lemmatization, stop or a combination of the two. Hence these three transformations do not have any increased benefit over taking the bare corpus. It is possible that the reason for this lies with the hashing itself, and that the output from hashing and the resulting dimensionality reduction is not affected by lemm or stop (and lemm and stop reduce dimensionality as well). One way to test this hypothesis is to perform this experiment without hashing; however, this would be impractical and too computationally expensive. Hence lemm and stop are good preprocessing steps to take in spam filtering, since they can reduce the dataset without reducing the accuracy of both SVM and LR, which are discriminative models. 

In reducing order of accuracy, NB, a generative model, performs best on Bare followed by lem, stop and lemm stop. However, NB is also a low-bias high variance model, and the variation in the results in not great enough to suggest that lemm and stop have a big effect on accuracy. Hence it is recommended that these pre-processing steps are still performed when using NB, as like mentioned before they reduce the dimensionality and the size of the dataset. 

## Writing Tool

The best strategy to increase writing speed while maintaining quality is twofold:
1. Write a bad first draft and do so as quickly as possible - do not overthink and worry about writing quality.
2. Be ruthless in editing what you just wrote.

After completing this NLP project and learning about stemming and lemmatization, I thought of a tool that can be used to help edit a first draft. Specifically, this tool would use stemming and lemmatization to quickly find repetitions in text (not including stop words). The tool would be used from the command line, and works as follows:
1. run the python script
2. after the prompt, copy paste the text you want to edit
3. press enter and get the count of repeated words.

The script for the tool in included below. I used to paragraph above as an example. 

In [24]:
import nltk, re, string

#nltk.download('wordnet')

# user inputs paragraph
pargraph_str = input('Copy paste paragraph: \n--------------------- \n ')
pargraph_str = pargraph_str.lower()

# removes punctuation and creates list of words
pargraph_str = pargraph_str.replace(',', ' ')
pargraph_str = pargraph_str.replace('.', ' ')
pargraph_str = pargraph_str.replace("'", '')
pargraph_str_split = pargraph_str.split()

# list of stop words to ignore, can add to this list
stop_words = ['a','the','and','this','of','is','to','as','in','by','an','into','it','be','on','that','at']

# removes stopwords from list
new_words = [word for word in pargraph_str_split if (word not in stop_words) 
and (word.isdigit()==False) # to remove digits
and (word != '=') # to remove equations
and ('(' not in word)] # to remove citations

# stemmers
ps = PorterStemmer()
lan = LancasterStemmer()

# lemmatizer
lm = WordNetLemmatizer()

# empty lists
new_lst_stem = []
new_lst_lem = []
new_lst_lan = []

# creates list with only stem words
for i in new_words:
    new_lst_stem.append(ps.stem(i))

for i in new_words:
    new_lst_lem.append(lm.lemmatize(i))

for i in new_words:
    new_lst_lan.append(lan.stem(i))

#counter
counter_stem = dict(Counter(new_lst_stem))
counter_lem = dict(Counter(new_lst_lem))
counter_lan = dict(Counter(new_lst_lan))

# remove when value of dict/counter is equal to 1
new_dict_stem = {key:val for key, val in counter_stem.items() if val != 1}
new_dict_lem = {key:val for key, val in counter_lem.items() if val != 1}
new_dict_lan = {key:val for key, val in counter_lan.items() if val != 1}

# output
if (bool(new_dict_stem)==False) and (bool(new_dict_lem)==False) and (bool(new_dict_lan)==False):
	print('\n No repetitions found') # prints in case no repetitions found
else:
    print('--------------------- \n')
    print('Porter stem: \n')
    for k,v in new_dict_stem.items():
        print(k,v)
    print('\n')
    print('Lancaster stem:\n')
    for k,v in new_dict_lan.items():
        print(k,v)
    print('\n')
    print('Wornet lem:\n')
    for k,v in new_dict_lem.items():
        print(k,v)         

Copy paste paragraph: 
--------------------- 
 The best strategy to increase writing speed while maintaining quality is twofold: Write a bad first draft, and do so as quickly as possible - do not overthink and worry about writing quality. Be ruthless in editing what you just wrote. After completing this NLP project and learning and stemming and lemmatization, I thought of a tool that can be used to help edit a first draft. Specifically, this tool would use stemming and lemmatization to quickly find repetitions in text (not including stop words). The tool would be used from the command line, and works as follows: run the python script after the prompt, copy paste the text you want to edit press enter and find the repeated words.
--------------------- 

Porter stem: 

write 3
qualiti 2
first 2
draft 2
do 2
quickli 2
edit 3
you 2
after 2
stem 2
lemmat 2
tool 3
use 3
would 2
find 2
text 2


Lancaster stem:

writ 3
qual 2
first 2
draft 2
do 2
quick 2
edit 3
you 2
aft 2
stem 2
lem 2
tool 3
u

I can now use the information above to rewrite the last paragraph and reduce the number of repetitions. 

## Appendix - Experiment Results

In [16]:
folder = 'lingspam_public/bare'
N = numpy.array([3,30,300,3000]) 
print('\nEXPERIMENT 1: Testing different vector sizes')
results = []
for n in N:
    print('N = {}'.format(n))
    A = trainTestFolder(folder,n)
    results.append(A)
    
n = 3000
typeFolders = ['lingspam_public/bare','lingspam_public/stop','lingspam_public/lemm','lingspam_public/lemm_stop']
print('\nEXPERIMENT 2: Testing different data types')
for folder in typeFolders:
    print('Path = {}'.format(folder))
    B = trainTestFolder(folder,n)
    results.append(B)


EXPERIMENT 1: Testing different vector sizes
N = 3
Accuracy 83.3% (data items: 15158, correct: 12622)
Accuracy 83.4% (data items: 1734, correct: 1446)
Accuracy 83.3% (data items: 15158, correct: 12622)
Accuracy 83.4% (data items: 1734, correct: 1446)
Accuracy 83.3% (data items: 15158, correct: 12622)
Accuracy 83.4% (data items: 1734, correct: 1446)
N = 30
Accuracy 85.9% (data items: 15158, correct: 13026)
Accuracy 85.1% (data items: 1734, correct: 1476)
Accuracy 83.3% (data items: 15158, correct: 12622)
Accuracy 83.4% (data items: 1734, correct: 1446)
Accuracy 83.3% (data items: 15158, correct: 12622)
Accuracy 83.4% (data items: 1734, correct: 1446)
N = 300
Accuracy 100.0% (data items: 15158, correct: 15158)
Accuracy 92.7% (data items: 1734, correct: 1608)
Accuracy 86.6% (data items: 15158, correct: 13130)
Accuracy 85.5% (data items: 1734, correct: 1482)
Accuracy 83.3% (data items: 15158, correct: 12622)
Accuracy 83.4% (data items: 1734, correct: 1446)
N = 3000
Accuracy 100.0% (data i